<a href="https://colab.research.google.com/github/KenyanGeek/Fraud-Detections-using-Fast.ai-Unbalanced-Data/blob/master/Xente_Fraud_Detection_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Fraud at Xende

In [0]:
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.

In [0]:
from google.colab import files
uploaded = files.upload()


Saving training.csv to training.csv


In [0]:
import io


df_train = pd.read_csv(io.BytesIO(uploaded['training.csv']))

df_train


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0
5,TransactionId_23223,BatchId_25954,AccountId_1078,SubscriptionId_4238,CustomerId_1432,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,2000.0,2000,2018-11-15T03:35:10Z,2,0
6,TransactionId_118063,BatchId_118460,AccountId_2442,SubscriptionId_1980,CustomerId_2858,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,10000.0,10000,2018-11-15T03:44:31Z,4,0
7,TransactionId_100640,BatchId_38561,AccountId_4841,SubscriptionId_3829,CustomerId_2858,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-500.0,500,2018-11-15T03:45:13Z,2,0
8,TransactionId_51905,BatchId_93774,AccountId_272,SubscriptionId_4731,CustomerId_598,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,500.0,500,2018-11-15T04:14:59Z,2,0
9,TransactionId_130161,BatchId_82409,AccountId_710,SubscriptionId_920,CustomerId_1053,UGX,256,ProviderId_1,ProductId_15,financial_services,ChannelId_3,600.0,600,2018-11-15T04:31:48Z,2,0


In [0]:
dep_var = 'FraudResult'


cat_names = ['TransactionId','BatchId','AccountId','SubscriptionId','CustomerId','CurrencyCode','CountryCode','ProviderId','ProductId','ProductCategory','ChannelId','TransactionStartTime','PricingStrategy']
cont_names = ['Amount','Value']
procs = [FillMissing, Categorify, Normalize]

In [0]:
from google.colab import files
uploaded = files.upload()


Saving test.csv to test (1).csv


In [0]:
import io


df_test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

test = TabularList.from_df(df_test, cat_names=cat_names, cont_names=cont_names, procs=procs) 


In [0]:
data = (TabularList.from_df(df_train, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_rand_pct(0.2)
                           .label_from_df(cols=dep_var)
                           .add_test(test, label = 'FraudResult')
                           .databunch())


In [0]:
data.show_batch(rows=10)

TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,TransactionStartTime,PricingStrategy,Amount,Value,target
TransactionId_112805,BatchId_10532,AccountId_4841,SubscriptionId_3829,CustomerId_4431,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,2019-02-10T10:11:38Z,2,-0.0555,-0.0807,0
TransactionId_84723,BatchId_111367,AccountId_1955,SubscriptionId_4208,CustomerId_2349,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,2018-11-23T15:24:29Z,2,-0.0468,-0.0729,0
TransactionId_505,BatchId_107288,AccountId_1060,SubscriptionId_2067,CustomerId_1413,UGX,256,ProviderId_1,ProductId_10,airtime,ChannelId_3,2019-02-10T14:40:14Z,4,-0.0221,-0.0481,0
TransactionId_80804,BatchId_78561,AccountId_4841,SubscriptionId_3829,CustomerId_3952,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,2018-12-29T09:40:34Z,2,-0.0559,-0.0803,0
TransactionId_754,BatchId_111734,AccountId_231,SubscriptionId_3603,CustomerId_556,UGX,256,ProviderId_1,ProductId_3,airtime,ChannelId_3,2018-12-14T19:52:29Z,4,0.0274,0.0015,0
TransactionId_22872,BatchId_86428,AccountId_3507,SubscriptionId_2644,CustomerId_3944,UGX,256,ProviderId_6,ProductId_4,airtime,ChannelId_3,2018-11-23T05:01:12Z,2,-0.0510,-0.0770,0
TransactionId_82200,BatchId_95761,AccountId_4841,SubscriptionId_3829,CustomerId_1987,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,2019-01-24T19:10:11Z,2,-0.0569,-0.0793,0
TransactionId_110907,BatchId_118916,AccountId_383,SubscriptionId_3299,CustomerId_713,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2018-12-07T16:57:12Z,2,0.1099,0.0841,0
TransactionId_90227,BatchId_97652,AccountId_2288,SubscriptionId_3596,CustomerId_2701,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,2019-02-06T19:48:36Z,2,-0.0468,-0.0729,0
TransactionId_107296,BatchId_77597,AccountId_318,SubscriptionId_3087,CustomerId_647,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,2018-12-31T12:50:37Z,2,-0.0468,-0.0729,0


In [0]:
learn = tabular_learner(data, layers=[1000,500], metrics=accuracy)

In [0]:
learn.fit(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.002729,0.040145,0.997752,1:03:30


## Inference

## Here i can predict with 99.77% accuracy

In [0]:
learn.predict(row)

(Category Good, tensor(1), tensor([0.2381, 0.7619]))

In [0]:

probs = learn.get_preds(ds_type=DatasetType.Test)[0]

def probs2class(item): 
    return max(range(len(item)), key=item.__getitem__) 

test_df = pd.DataFrame({'TransactionId': df_test['TransactionId'], 'target': list(map(probs2class, probs))})
test_df.to_csv('xende predictions.csv', header=True, index=False)

